In [1]:
import os
import pandas as pd

# os.listdir('../')

In [2]:
train = pd.read_csv("../235745_parking_data/train.csv")
test = pd.read_csv("../235745_parking_data/test.csv")
sub = pd.read_csv("../235745_parking_data/sample_submission.csv")
age = pd.read_csv("../235745_parking_data/age_gender_info.csv")

train.shape, test.shape, sub.shape, age.shape

((2952, 15), (1022, 14), (150, 2), (16, 23))

In [3]:
mapper = {"도보 10분거리 내 지하철역 수(환승노선 수 반영)":"10분내지하철수", 
         "도보 10분거리 내 버스정류장 수":"10분내버스정류장수"}

train.rename(columns=mapper, inplace=True)
test.rename(columns=mapper, inplace=True)

print(train.columns)
print(test.columns)

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '10분내지하철수', '10분내버스정류장수', '단지내주차면수', '등록차량수'],
      dtype='object')
Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '10분내지하철수', '10분내버스정류장수', '단지내주차면수'],
      dtype='object')


## 테스트 셋 제외 데이터
테스트셋에서 평가 제외되는 데이터는 'C2675'(2번 사항에 해당), 'C2335', 'C1327'(3번 사항에 해당) 3개 단지입니다.

In [4]:
train.loc[ ((train['단지코드']=='C2675') | 
           (train['단지코드']=='C2335') |
           (train['단지코드']=='C1327') ) , :]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수


In [5]:
test.loc[ ((test['단지코드']=='C2675') | 
           (test['단지코드']=='C2335') |
           (test['단지코드']=='C1327') ) , :].head(3)

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
579,C2675,512,아파트,경기도,국민임대,36.65,130,9.0,A,18476000,154790,0.0,3.0,1016.0
580,C2675,512,아파트,경기도,국민임대,46.90,44,9.0,A,34082000,232200,0.0,3.0,1016.0
581,C2675,512,아파트,경기도,국민임대,46.90,80,9.0,A,34082000,232200,0.0,3.0,1016.0


In [6]:
# 테스트 셋에서 세 개의 데이터 제거

test = test.loc[ ~((test['단지코드']=='C2675') | 
           (test['단지코드']=='C2335') |
           (test['단지코드']=='C1327') ) , :]
test.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
0,C1072,754,아파트,경기도,국민임대,39.79,116,14.0,H,22830000,189840,0.0,2.0,683.0
1,C1072,754,아파트,경기도,국민임대,46.81,30,14.0,A,36048000,249930,0.0,2.0,683.0
2,C1072,754,아파트,경기도,국민임대,46.90,112,14.0,H,36048000,249930,0.0,2.0,683.0
3,C1072,754,아파트,경기도,국민임대,46.90,120,14.0,H,36048000,249930,0.0,2.0,683.0
4,C1072,754,아파트,경기도,국민임대,51.46,60,14.0,H,43497000,296780,0.0,2.0,683.0


In [7]:
# 확인
test.loc[ ((test['단지코드']=='C2675') | 
           (test['단지코드']=='C2335') |
           (test['단지코드']=='C1327') ) , :]


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수


## ERROR 1 처리
- 문제 : 전용면적별 세대수 합계와 총세대수가 일치하지 않는 경우  
- 원인 : 차이는 분양상가 또는 분양아파트가 임대단지 안에 혼합되어 발생한 것으로, 분양 완료된 상가, 아파트는 관리대상에서 제외되어 전용면적별 세대수에서는 포함되지 않았으나 총세대수에는 포함
- 분포 (*error_description.ipynb 파일 확인*): 
    - 차이는 최소 1세대부터 최대 452세대까지이며, 그 가운데 48개 단지(83%) 는 차이가 1~14세대, 나머지 10개 단지(17%) 는 차이가 94~452세대 사이에 분포
    - 차이가 14세대 이하인 48개 단지는 __분양상가__, 차이가 94~452세대에 분포하는 10개 단지는 __분양아파트__ 혼합단지인 것으로 확인

>  ※ 차이가 94~452세대인 10개 단지(크기순) : ['C1490', 'C2497', 'C2620', 'C1344', 'C1024', 'C2470', 'C1206', 'C1740', 'C2405', 'C1804’]  
> ※ 별첨 : 차이가 14세대 이하인 48개 단지 - ['C1925', 'C1312', 'C2013', 'C1424', 'C2520', 'C2319', 'C1850', 'C1068', 'C2644', 'C2156', C2453', 'C1910', 'C2139', 'C2508', 'C1695', 'C2556', 'C2362', 'C2568', 'C2245', 'C2549', 'C1584', 'C2298', 'C2225', 'C1218', 'C1970', C1732', 'C2433', 'C1894', 'C1156', 'C2142', 'C2186', 'C2411', 'C1812', 'C1030', 'C1749', 'C1349', 'C2043', 'C1229', 'C2363', 'C1414', C2174', 'C2404', 'C1683', 'C1038', 'C2456', 'C1266', 'C1267', 'C2189']

- 처리 : 차이가 14세대 이하인 단지(grp2)는 '분양상가'로, 차이가 94~452세대인 10개 단지(grp1)는 '분양아파트'로, 나머지는 '나머지'로 라벨링을 한 칼럼을 하나 만든다.

In [8]:
def error1(x):
    if x in grp1 :
        return '분양아파트'
    elif x in grp2 :
        return '분양상가'
    else :
        return '나머지'

grp1 = ['C1490', 'C2497', 'C2620', 'C1344', 'C1024', 'C2470', 'C1206', 'C1740', 'C2405', 'C1804']
grp2 = ['C1925', 'C1312', 'C2013', 'C1424', 'C2520', 'C2319', 'C1850', 'C1068', 'C2644', 'C2156', 
        'C2453', 'C1910', 'C2139', 'C2508', 'C1695', 'C2556', 'C2362', 'C2568', 'C2245', 'C2549', 
        'C1584', 'C2298', 'C2225', 'C1218', 'C1970', 'C1732', 'C2433', 'C1894', 'C1156', 'C2142', 
        'C2186', 'C2411', 'C1812', 'C1030', 'C1749', 'C1349', 'C2043', 'C1229', 'C2363', 'C1414', 
        'C2174', 'C2404', 'C1683', 'C1038', 'C2456', 'C1266', 'C1267', 'C2189']

train['ERROR1'] = train['단지코드'].apply(error1)
test['ERROR1'] = test['단지코드'].apply(error1)

print(train.shape, test.shape)
print(train['ERROR1'].value_counts())
print(test['ERROR1'].value_counts())

(2952, 16) (1008, 15)
나머지      2730
분양상가      171
분양아파트      51
Name: ERROR1, dtype: int64
나머지      857
분양상가     147
분양아파트      4
Name: ERROR1, dtype: int64


## ERROR 2 처리
- 문제 : 동일한 단지임에도 불구하고 단지코드가 2개 부여됨
- 원인 : 1번과 마찬가지로 분양과 임대가 혼합되어 발생
- 동일한 단지에 코드가 2개 부여된 단지 코드 (3쌍) : 
|train set|train/test set|
|:--:|:--:|
|['C2085', 'C1397'], <br> ['C2431', 'C1649']| ['C1036'/ 'C2675'] |
- 참고 :  주차면수는 하나의 단지임을 전제로 산정한 것이고 총 세대수는 두 개 단지의 합계  
 다만 등록차량대수는 ['C2085', 'C1397'] 단지의 경우 동일 수치, ['C2431', 'C1649']] 단지의 경우 두 개 단지 합계입니다. 
![error](https://user-images.githubusercontent.com/38090151/126156807-4c2f82c1-1e37-4d86-8329-ea202670e42c.png)

> ERROR2 데이터를 삭제할 경우 에러 데이터가 가지고 있는 단지코드 이외의 피처 레이블들을 사용할 수 없기 때문에 삭제하지 않고 그대로 둔다.

### 'C2085', 'C1397' 처리

In [9]:
# 총세대수 처리
train.loc[train['단지코드']=='C2085', "총세대수"] = 1339
train.loc[train['단지코드']=='C1397', "총세대수"] = 1339

# 'C2085', 'C1397' shape 확인
print( train.loc[train['단지코드']=='C2085',:].shape )
print( train.loc[train['단지코드']=='C1397',:].shape )

# 단지코드 처리
train.loc[train['단지코드']=='C2085', "단지코드"] = 'N2085'
train.loc[train['단지코드']=='C1397', "단지코드"] = 'N2085'

# 처리 후 확인
print(train[train['단지코드']=='N2085'].shape)

train[train['단지코드']=='N2085']

(8, 16)
(6, 16)
(14, 16)


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수,ERROR1
2043,N2085,1339,아파트,서울특별시,장기전세,23.32,51,9.0,A,87444000,-,0.0,3.0,1590.0,1595.0,나머지
2044,N2085,1339,아파트,서울특별시,장기전세,46.79,4,9.0,A,174888000,-,0.0,3.0,1590.0,1595.0,나머지
2045,N2085,1339,아파트,서울특별시,장기전세,46.91,69,9.0,A,174888000,-,0.0,3.0,1590.0,1595.0,나머지
2046,N2085,1339,아파트,서울특별시,장기전세,51.96,24,9.0,A,194562000,-,0.0,3.0,1590.0,1595.0,나머지
2047,N2085,1339,아파트,서울특별시,장기전세,51.99,80,9.0,A,194562000,-,0.0,3.0,1590.0,1595.0,나머지
2048,N2085,1339,아파트,서울특별시,장기전세,59.93,142,9.0,A,216423000,-,0.0,3.0,1590.0,1595.0,나머지
2186,N2085,1339,아파트,서울특별시,공공임대(10년),51.96,63,0.0,A,47001000,602270,0.0,3.0,1590.0,1595.0,나머지
2187,N2085,1339,아파트,서울특별시,공공임대(10년),51.99,41,0.0,A,47001000,602270,0.0,3.0,1590.0,1595.0,나머지
2188,N2085,1339,아파트,서울특별시,공공임대(10년),59.93,315,0.0,A,61210000,695170,0.0,3.0,1590.0,1595.0,나머지
2189,N2085,1339,아파트,서울특별시,공공임대(분납),74.80,208,0.0,A,120611000,293490,0.0,3.0,1590.0,1595.0,나머지


## 'C2431', 'C1649' 처리

In [10]:
# 총세대수 처리
train.loc[train['단지코드']=='C2431', "총세대수"] = 1047
train.loc[train['단지코드']=='C1649', "총세대수"] = 1047

# 등록차량대수 처리
train.loc[train['단지코드']=='C2431', "등록차량수"] = 1214
train.loc[train['단지코드']=='C1649', "등록차량수"] = 1214

# 'C2431', 'C1649' shape 확인
print( train.loc[train['단지코드']=='C2431',:].shape )
print( train.loc[train['단지코드']=='C1649',:].shape )

# 단지코드 처리
train.loc[train['단지코드']=='C2431', "단지코드"] = 'N2431'
train.loc[train['단지코드']=='C1649', "단지코드"] = 'N2431'

# 처리 후 확인
print(train[train['단지코드']=='N2431'].shape)

train[train['단지코드']=='N2431']

(2, 16)
(4, 16)
(6, 16)


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수,ERROR1
2315,N2431,1047,아파트,경상남도,공공임대(10년),74.97,80,15.0,A,46000000,456000,NaN,NaN,1066.0,1214.0,나머지
2316,N2431,1047,아파트,경상남도,공공임대(10년),84.95,124,15.0,A,57000000,462000,NaN,NaN,1066.0,1214.0,나머지
2317,N2431,1047,아파트,경상남도,공공임대(10년),84.96,289,15.0,A,57000000,462000,NaN,NaN,1066.0,1214.0,나머지
2318,N2431,1047,아파트,경상남도,공공임대(10년),84.98,82,15.0,A,57000000,462000,NaN,NaN,1066.0,1214.0,나머지
2372,N2431,1047,아파트,경상남도,국민임대,36.77,272,16.0,A,11217000,233330,0.0,2.0,1066.0,1214.0,나머지
2373,N2431,1047,아파트,경상남도,국민임대,46.78,200,16.0,A,24389000,303220,0.0,2.0,1066.0,1214.0,나머지


## 'C1036', 'C2675' 처리
- test 셋의 'C2675'는 '테스트 셋에서 제외 데이터'에 포함되어 있어서 이미 제거됨

In [11]:
# train 셋에서 'C1036' 총세대수 처리
train.loc[train['단지코드']=='C1036',"총세대수"] = 1254

# 'C1036', 'C2675' shape 확인
print( train.loc[train['단지코드']=='C1036',:].shape )

# 단지코드 처리
train.loc[ train['단지코드']=='C1036',  "단지코드" ] = 'N1036'

# 처리 후 확인
print(train.loc[train['단지코드']=='N1036'].shape)
train[train['단지코드']=='N1036']

(7, 16)
(7, 16)


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수,ERROR1
1502,N1036,1254,아파트,경기도,국민임대,36.65,135,13.0,A,18476000,154790,0.0,3.0,1016.0,1279.0,나머지
1503,N1036,1254,아파트,경기도,국민임대,46.90,45,13.0,A,34082000,232200,0.0,3.0,1016.0,1279.0,나머지
1504,N1036,1254,아파트,경기도,국민임대,46.90,171,13.0,A,34082000,232200,0.0,3.0,1016.0,1279.0,나머지
1505,N1036,1254,아파트,경기도,국민임대,46.90,81,13.0,A,34082000,232200,0.0,3.0,1016.0,1279.0,나머지
1506,N1036,1254,아파트,경기도,국민임대,46.90,90,13.0,A,34082000,232200,0.0,3.0,1016.0,1279.0,나머지
1507,N1036,1254,아파트,경기도,국민임대,51.93,184,13.0,A,43819000,275900,0.0,3.0,1016.0,1279.0,나머지
1508,N1036,1254,아파트,경기도,국민임대,51.93,36,13.0,A,43819000,275900,0.0,3.0,1016.0,1279.0,나머지


## ERROR 3
- 문제 : 단지코드 등 기입 실수로 데이터 정제 과정에서 매칭 오류 발생  
> ※ 실수가 발생한 단지 코드 (9개 단지) : 

|test set| train set|
|:--:|:--:|
|'C2335', 'C1327' | 'C1095', 'C2051', 'C1218', 'C1894', 'C2483', 'C1502', 'C1988'|

- 'C2335', 'C1327'는 '테스트 셋 제외 데이터'에 포함되어 있어서 이미 제거됨

In [12]:
# 처리할 데이터의 갯수 확인
print(train[(train['단지코드']=='C1095')|
      (train['단지코드']=='C2051')|
      (train['단지코드']=='C1218')|
      (train['단지코드']=='C1894')|
      (train['단지코드']=='C2483')|
      (train['단지코드']=='C1502')|
      (train['단지코드']=='C1988')
     ].shape)

# 해당 데이터의 단지코드를 'ERR04' 형식으로 처리
train.loc[ train['단지코드']=='C1095',  "단지코드" ] = 'ERR04_1095'
train.loc[ train['단지코드']=='C2051',  "단지코드" ] = 'ERR04_2051'
train.loc[ train['단지코드']=='C1218',  "단지코드" ] = 'ERR04_1218'
train.loc[ train['단지코드']=='C1894',  "단지코드" ] = 'ERR04_1894'
train.loc[ train['단지코드']=='C2483',  "단지코드" ] = 'ERR04_2483'
train.loc[ train['단지코드']=='C1502',  "단지코드" ] = 'ERR04_1502'
train.loc[ train['단지코드']=='C1988',  "단지코드" ] = 'ERR04_1988'

# 처리 후 갯수 확인
print(train.loc[ train['단지코드'].str.contains('ERR'), :].shape)
train.loc[ train['단지코드'].str.contains('ERR'), :]


(56, 16)
(56, 16)


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수,ERROR1
0,ERR04_2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0,나머지
1,ERR04_2483,900,아파트,경상북도,국민임대,39.72,15,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0,나머지
2,ERR04_2483,900,아파트,경상북도,국민임대,51.93,385,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0,나머지
3,ERR04_2483,900,아파트,경상북도,국민임대,51.93,15,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0,나머지
4,ERR04_2483,900,아파트,경상북도,국민임대,51.93,41,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0,나머지
5,ERR04_2483,900,아파트,경상북도,국민임대,51.95,89,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0,나머지
6,ERR04_2483,900,아파트,경상북도,국민임대,51.95,135,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0,나머지
7,ERR04_2483,900,아파트,경상북도,국민임대,59.88,86,38.0,A,30357000,214270,0.0,3.0,1425.0,1015.0,나머지
1228,ERR04_1988,475,아파트,전라남도,국민임대,36.63,200,12.0,A,12026000,87940,0.0,2.0,722.0,402.0,나머지
1229,ERR04_1988,475,아파트,전라남도,국민임대,36.63,43,12.0,A,12026000,87940,0.0,2.0,722.0,402.0,나머지


## csv, xlsx 파일 내보내기

In [13]:
path ='./data/'

train_df = train.copy()
train_df_errx = train.loc[~ train['단지코드'].str.contains('ERR'), :]
test_df = test.copy()

train_df.to_csv(path+"1_train.csv", index=False)
train_df_errx.to_csv(path+"1_train_errx.csv", index=False)
test_df.to_csv(path+"1_test.csv", index=False)

train_df.to_excel(path+"1_train.xlsx", index=False)
train_df_errx.to_excel(path+"1_train_errx.xlsx", index=False)
test_df.to_excel(path+"1_test.xlsx", index=False)

os.listdir(path)

['2_test.xlsx',
 '3_train.csv',
 'age_gender_info.csv',
 '.DS_Store',
 '2_train.xlsx',
 '2_train_errx.xlsx',
 '3_test.xlsx',
 'test.csv',
 '2_train.csv',
 '1_test.csv',
 '3_train.xlsx',
 '1_train.csv',
 '1_train_errx.xlsx',
 '2_train_errx.csv',
 'train.csv',
 '1_test.xlsx',
 '1_train.xlsx',
 '2_test.csv',
 '3_test.csv',
 'save',
 '1_train_errx.csv',
 'sample_submission.csv']

> 에러 처리 완료